In [1]:
import matplotlib.pyplot as plt
import random

from matplotlib.colors import ListedColormap
from sklearn import datasets

import numpy as np

In [128]:
# сгенерируем данные
classification_data, classification_labels = datasets.make_classification(n_samples=10000, n_features = 2, n_informative = 2, 
                                                      n_classes = 2, n_redundant=0, 
                                                      n_clusters_per_class=1, random_state=5)

In [129]:
class Arbitrator:

    def __init__(self, nodes_n, leaf_n, deep_tree, potential_leaves):
        self.nodes_n = nodes_n
        self.leaf_n = leaf_n
        self.deep_tree = deep_tree
        self.potential_leaves = potential_leaves
        self.leaf_know = 0  # используется для проверки кол-ва листьев, для работы основной рекурсии не требуется
        self.node_know = 0  # аналогично, узлы, впрочем кол-во узлов равно leaf_n - 1
        self.deep_know = 0  # аналогично, глубина

In [130]:
# Реализуем класс узла

class Node:
    
    def __init__(self, index, t, true_branch, false_branch):
        self.index = index  # индекс признака, по которому ведется сравнение с порогом в этом узле
        self.t = t  # значение порога
        self.true_branch = true_branch  # поддерево, удовлетворяющее условию в узле
        self.false_branch = false_branch  # поддерево, не удовлетворяющее условию в узле

In [131]:
#  И класс терминального узла (листа)
class Leaf:
    
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels
        self.prediction = self.predict()
        
    def predict(self):
        # подсчет количества объектов разных классов
        return np.mean(self.labels)

In [132]:
# Расчет критерия Джини

def gini(labels):
    #  подсчет количества объектов разных классов
    impurity = np.var(labels)
    return impurity

In [133]:
#  Расчет качества

def quality(left_labels, right_labels, current_gini):

    # доля выбоки, ушедшая в левое поддерево
    p = float(left_labels.shape[0]) / (left_labels.shape[0] + right_labels.shape[0])
    
    return current_gini - p * gini(left_labels) - (1 - p) * gini(right_labels)

In [134]:
# Разбиение датасета в узле

def split(data, labels, index, t):
    
    left = np.where(data[:, index] <= t)
    right = np.where(data[:, index] > t)
        
    true_data = data[left]
    false_data = data[right]
    true_labels = labels[left]
    false_labels = labels[right]
        
    return true_data, false_data, true_labels, false_labels

In [135]:
#  Нахождение наилучшего разбиения

def find_best_split(data, labels):
    
    #  обозначим минимальное количество объектов в узле
    min_leaf = 5

    current_gini = gini(labels)

    best_quality = 0
    best_t = None
    best_index = None
    
    n_features = data.shape[1]
    
    
    for index in range(n_features):
        # будем проверять только уникальные значения признака, исключая повторения
        t_values = np.unique([row[index] for row in data])
        
        for t in t_values:
            true_data, false_data, true_labels, false_labels = split(data, labels, index, t)
            #  пропускаем разбиения, в которых в узле остается менее 5 объектов
            if len(true_data) < min_leaf or len(false_data) < min_leaf:
                continue
            current_quality = quality(true_labels, false_labels, current_gini)
            
            #  выбираем порог, на котором получается максимальный прирост качества
            if current_quality > best_quality:
                best_quality, best_t, best_index = current_quality, t, index

    return best_quality, best_t, best_index

In [153]:
# Построение дерева с помощью рекурсивной функции
def build_tree(data, labels,  cur_level, node =0, leaf = 0, deep = 0):
    print('new_tree:', cur_level, print(cur_level > deep))
    flag = False
    print(f'leaves: {ar.leaf_n}', f'deep: {ar.deep_tree}',  f'potential leaves: {ar.potential_leaves}', f'leafs: {ar.leaf_n}')
    if deep != 0 or leaf != 0 or node != 0:
        if deep != 0:
            if cur_level >= deep:
                ar.leaf_n = ar.leaf_n + 1
                return Leaf(data, labels)
            else:
                if leaf != 0:
                    if (ar.leaf_n+ar.potential_leaves) > leaf:
                        ar.leaf_n = ar.leaf_n + 1
                        return Leaf(data, labels)
                    else:
                        if node != 0:
                            if ar.nodes_n > node:
                                ar.leaf_n = ar.leaf_n + 1
                                return Leaf(data, labels)
                            else:
                                flag = True
                        else:
                            flag =True
    else: 
        flag = True
    if flag:
        #  Базовый случай - прекращаем рекурсию, когда нет прироста в качества
        quality, t, index = find_best_split(data, labels)
        if quality == 0:
            ar.leaf_n = ar.leaf_n + 1
            return Leaf(data, labels)
        else:
            ar.nodes_n = ar.nodes_n + 1
            true_data, false_data, true_labels, false_labels = split(data, labels, index, t)
            # Рекурсивно строим два поддерева
            level = cur_level + 1
            ar.deep_tree = max(level, ar.deep_tree)
            ar.potential_leaves = ar.deep_tree 
            print('true', level)
            true_branch = build_tree(true_data, true_labels, cur_level=level, 
                                     node=node, leaf=leaf, deep=deep)
            print('false', level)
            false_branch = build_tree(false_data, false_labels, cur_level=level, 
                                      node=node, leaf=leaf, deep=deep)

            # Возвращаем класс узла со всеми поддеревьями, то есть целого дерева
            return Node(index, t, true_branch, false_branch)

In [154]:
def classify_object(obj, node):

    #  Останавливаем рекурсию, если достигли листа
    if isinstance(node, Leaf):
        answer = node.prediction
        return answer

    if obj[node.index] <= node.t:
        return classify_object(obj, node.true_branch)
    else:
        return classify_object(obj, node.false_branch)

In [155]:
def predict(data, tree):
    
    classes = []
    for obj in data:
        prediction = classify_object(obj, tree)
        classes.append(prediction)
    return classes

In [156]:
from sklearn import model_selection

train_data, test_data, train_labels, test_labels = model_selection.train_test_split(classification_data, 
                                                                                     classification_labels, 
                                                                                     test_size = 0.3,
                                                                                     random_state = 1)

In [159]:
# Построим дерево по обучающей выборке
ar = Arbitrator(0,0,0,0)
my_tree = build_tree(train_data, train_labels, cur_level = 0,  leaf =7, deep = 3 )

False
new_tree: 0 None
leaves: 0 deep: 0 potential leaves: 0 leafs: 0
true 1
False
new_tree: 1 None
leaves: 0 deep: 1 potential leaves: 1 leafs: 0
true 2
False
new_tree: 2 None
leaves: 0 deep: 2 potential leaves: 2 leafs: 0
true 3
False
new_tree: 3 None
leaves: 0 deep: 3 potential leaves: 3 leafs: 0
false 3
False
new_tree: 3 None
leaves: 1 deep: 3 potential leaves: 3 leafs: 1
false 2
False
new_tree: 2 None
leaves: 2 deep: 3 potential leaves: 3 leafs: 2
true 3
False
new_tree: 3 None
leaves: 2 deep: 3 potential leaves: 3 leafs: 2
false 3
False
new_tree: 3 None
leaves: 3 deep: 3 potential leaves: 3 leafs: 3
false 1
False
new_tree: 1 None
leaves: 4 deep: 3 potential leaves: 3 leafs: 4
true 2
False
new_tree: 2 None
leaves: 4 deep: 3 potential leaves: 3 leafs: 4
true 3
False
new_tree: 3 None
leaves: 4 deep: 3 potential leaves: 3 leafs: 4
false 3
False
new_tree: 3 None
leaves: 5 deep: 3 potential leaves: 3 leafs: 5
false 2
False
new_tree: 2 None
leaves: 6 deep: 3 potential leaves: 3 leafs: 6


In [161]:
# Напечатаем ход нашего дерева
def print_tree(node, spacing=""):

    # Если лист, то выводим его прогноз
    if isinstance(node, Leaf):
        print(spacing + "Прогноз:", node.prediction)
        return

    # Выведем значение индекса и порога на этом узле
    print(spacing + 'Индекс', str(node.index))
    print(spacing + 'Порог', str(node.t))

    # Рекурсионный вызов функции на положительном поддереве
    print (spacing + '--> True:')
    print_tree(node.true_branch, spacing + "  ")

    # Рекурсионный вызов функции на отрицательном поддереве
    print (spacing + '--> False:')
    print_tree(node.false_branch, spacing + "  ")
    
print_tree(my_tree)

Индекс 0
Порог -0.012660696990174802
--> True:
  Индекс 1
  Порог -1.9157918164675904
  --> True:
    Индекс 1
    Порог -2.0748877834842867
    --> True:
      Прогноз: 0.9927884615384616
    --> False:
      Прогноз: 0.7105263157894737
  --> False:
    Индекс 1
    Порог -1.7534373028375798
    --> True:
      Прогноз: 0.27058823529411763
    --> False:
      Прогноз: 0.014102162331557505
--> False:
  Индекс 1
  Порог -1.4744958270065416
  --> True:
    Индекс 0
    Порог 0.3212738116891256
    --> True:
      Прогноз: 0.6595238095238095
    --> False:
      Прогноз: 0.8356643356643356
  --> False:
    Прогноз: 0.9912203687445127
